#**Convolutional Neural Network**

*The notebook for deep introduction to Convolutional Neural Nets, implementations from scracth using Numpy and other advanced notes on written papers with summary of what I'll be learning.*

## Contents

- [1 - Introduction](#1)
- [2 - Packages](#2)


<a name='1'></a>
## 1 - Introduction

Short notes on Convolutional Neural Nets.